In [1]:
from nltk.tokenize import word_tokenize
import re
import collections
import pickle
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from gensim.test.utils import get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

In [50]:
import pandas as pd
import jieba
import json

In [24]:
trainpath = "D:\\Soong\\Soong\\textSummarization\\dataSet\\LCSTS\\CSV\\PART_II.csv"
validpath = "D:\\Soong\\Soong\\textSummarization\\dataSet\\LCSTS\\CSV\\PART_III.csv"

In [28]:
def get_list(path,clas):
    data=pd.read_csv(path)
    if clas=='article':
        index=data.columns.values[0]
    elif clas=='title':
        index=data.columns.values[1]
    listData=[]
    for text in data[index]:

        listData.append(text)
        
    return listData

In [37]:
def setMax(path,clas):
    data=pd.read_csv(path)
    if clas=='article':
        index=data.columns.values[0]
    elif clas=='title':
        index=data.columns.values[1]
    num_tokens = [ len(tokens) for tokens in data[index] ]
    num_tokens = np.array(num_tokens)
    
    # 取tokens平均值并加上两个tokens的标准差，
    # 假设tokens长度的分布为正态分布，则max_tokens这个值可以涵盖95%左右的样本
    max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
    max_tokens = int(max_tokens)
    return max_tokens


In [152]:
def build_dict(step):
    if step=="train":
        train_article=get_list(trainpath,'article')
        train_title=get_list(trainpath,'title')
        
        text=train_article+train_title
        words = list()
        for per in text:


            per=''.join(per)
        #     print(per)
            cut=jieba.lcut(per)
        #     print(cut)
            for precut in cut:
                precut = re.sub("”：,'[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "",str(precut))

                precut=precut.replace('”','').replace('：','').replace(',','').replace(',','').replace('[','').replace('[','')


                precut = re.sub(u"([^\u4e00-\u9fa5\u0030-\u0039\u0041-\u005a\u0061-\u007a])","",precut)

        #     # 结巴分词
        #     cut=jieba.lcut(per)

        #     cut=''.join(cut)
        #         print(cut)
        #         print(precut)
                words.append(precut)
        word_counter=[]
        for word in words:
            if word not in word_counter:
                word_counter.append(word)
        # word_counter = set(words)

        word_dict = dict()
        word_dict["<padding>"] = 0
        word_dict["<unk>"] = 1
        word_dict["<s>"] = 2
        word_dict["</s>"] = 3
        for  word in word_counter:

            word_dict[word] = len(word_dict)
        with open("D:\\Soong\\Soong\\textSummarization\\ChineseNewsSum\\zijide\\word_dict.pickle", "wb") as f:
            pickle.dump(word_dict, f)
    elif step == "valid":
        with open("D:\\Soong\\Soong\\textSummarization\\ChineseNewsSum\\zijide\\word_dict.pickle", "rb") as f:
            word_dict = pickle.load(f)
    reversed_dict = dict(zip(word_dict.values(), word_dict.keys()))
    

    
    article_max_len = setMax(trainpath,'article')
    summary_max_len = setMax(trainpath,'title')

    return word_dict, reversed_dict, article_max_len, summary_max_len

In [153]:
#train.py
word_dict, reversed_dict, article_max_len, summary_max_len = build_dict("train")

In [163]:
summary_max_len

25

In [168]:
def build_dataset(step, word_dict, article_max_len, summary_max_len):
    if step=="train":
        article=get_list(trainpath,'article')
        title=get_list(trainpath,'title')
        
        
    elif step == "valid":
        article = get_list(validpath,'article')
    else:
        raise NotImplementedError
    
    x = [jieba.lcut(''.join(d)) for d in article]
            
    x = [[word_dict.get(w, word_dict["<unk>"]) for w in d] for d in x]
    x = [d[:article_max_len] for d in x]
    x = [d + (article_max_len - len(d)) * [word_dict["<padding>"]] for d in x]
    
    if step == "valid":
        return x
    else:        
        y = [jieba.lcut(''.join(d)) for d in title]
        y = [[word_dict.get(w, word_dict["<unk>"]) for w in d] for d in y]
        y = [d[:(summary_max_len - 1)] for d in y]
        return x, y
    

In [169]:
#train.py
train_x, train_y = build_dataset("train", word_dict, article_max_len, summary_max_len)